# Reranker(리랭커)

Retriever가 찾아낸 문서들의 순위를 Reranker Model에 넣고 재조정한다. 

초기 검색결과의 정확도를 향샹시키는데 목표를 한다. Retriever가 대규모 문서 집합에서 관련성 잇는 후보 문서들을 빠르게 추출한 후, Reranker는 이 문서들을 정교하게 분석하여 최종순위 결정

**작동원리**

Retriever - Bi-Encoder: 문서와 query를 따로 놓고 encoding. 독립적 인코딩

Reranker - Cross Encoder: 문서와 query를 같이 놓고 encoding. 같이 인코딩



1. Retriever로부터 초기 검색결과를 입력받음
2. 쿼리와 쌍 후보 문서를 쌍으로 결합
3. 복잡한 모델(Transformer, Transformer등)을 사용하여 쿼리-문서-쌍의 관련성 평가
4. 평가 결과에 따라 문서들의 순위를 재조정
5. 최종적으로 재순위화된 결과를 출력

- retrieval와 reranker의 성능 비교
1. retrieval단계는 빠르지만 정확도가 떨어진다
2. reranker는 정확하지만 계산비용이 높다
3. 두방식을 결합해서 속도와 정확도를 동시에 얻자


- 왜 retriver를 쓰고 reranker를 써야할까?
1. cross encoding은 계산비용이 retrival보다 크다
2. bert와 같은 복잡한 언어 모델을 사용하므로 더 많은 연산이 필요하다
3. retrieval 단계에서 선별된 소수의 문서만 처리하므로 전체 시스템의 효율성 유지

즉!

`일반적으로 retrieval단계에서 상위 5~10단계 문서를 선택하고 reranker에 입력하여 최종 결과를 얻는다.`

`retriever에서 30개를 고르고, reranker에서 10개를 고르는 방식도 자주 쓰인다. `

reranker는 쓰면 확실히 좋다. 교차인코더 구조이다.
Reranker의 목표는 초기 검색결과의 정확도를 향상시키는것을 목표로 한다. Retriever가 대규모 문서 집합에서 관련성 있는 후보 문서들을 빠르게 추출한 후, Reranker는 이후 후보 문서들을더 정교하게 분석하여 최종적인 순위를 결정한다. 쿼리와 후보쌍

retriever를 쓰면 chunk를 잘라서 dense embedding으로 저장한다 이때 질문이 들어오면 청크와 질문의 유사도를 계산을해서 가장 높은 단락을 프롬프트에 넣어준다

이러면 질문과 유사도가 독립적으로 동작하는 문제가 발생한다

독립적으로 단순계산을 하게되는데, 이때 리랭커를 쓰면 크로스 인코더의 경우 질문과 컨텍스트를 동시에 넣어 계산한다. 청크와 질문을 쌍으로 넣어 계산을 처리한다. 그렇기 때문에 relevance를 높혀주는 것이다. 이때 문제점은 계산비용이 매우크다. 따라서 리트리버를 하고 후보군을 추려서, 이때 리랭커에 넣어 관련성 계산을 다시 하는 후처리가 트랜드이다.
계산비용이 크기때문에 바로쓰지 못하고, retriever를 통해서 관련있는 문서를 추출하고 reranker를 하는것이다.

리랭커는 딥러닝 모델이다. 임베딩과 같은맥락. BERT와 Transformer와 같은 복잡한 모델을 사용하기 때문에 많은 연산이 필요하다. 따라서 retriever를 쓰고 reranker를쓰자!

리랭커는 쓰는법이 어렵진 않지만 많이 안쓴다. 쓰자!


연산비용이 문제다. 그래서 API방식으로 많이쓴다.
BGE쓰면 좋다고한다 COHERE도 좋음 하루에 천건 무료 하지만 천건이 빨리 소비댐
문서에는 텍스트, 이미지, 테이블이 있는데 이 요소들을 따로따로 가져온다. 텍스트의 경우 제목, 줄글이 있는데, 마크다운 변환기를 안쓰면 구별이 안되는 경우가 많다.(pdf loader를 사용하는경우)
따라서 레이아웃 파싱 알고리즘을 써서 텍스트, 이미지, 테이블을 각각 요소화를 한다.
업스테이지 레이아웃 파싱 알고리즘이 좋다. 가격도 나쁘지 않다. api방식이다.(무료 크래딧 3만원) 무료버전은 아직 힘든다..
